In [1]:
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 10

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False
torch.set_num_threads(8)


def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "MNIST",
            'dataset': "BiasedMNIST",
            # 'dataset': "FashionMNIST",
            'random_class_idx': False,
            'method': "WA",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 5,
            # 'per_task_examples': np.inf,
            'per_task_examples': 10000,
            'per_task_memory_examples': 64,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 1.0,

            # 'tau': 0.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.01,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:7' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.00,
            'lambda': .0,
            'lambda_old': 0.0,
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/methods={params['method']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/demo/dataset=BiasedMNIST/methods=WA/seed=10_epoch=5_lr=0.01_alpha=0.0_tau=1.0


In [2]:
from datasets import MNIST
from datasets import FashionMNIST
from datasets import BiasedMNIST
from datasets import CIFAR10, CIFAR100

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                    per_task_memory_examples=params['per_task_memory_examples'],
                    per_task_examples = params['per_task_examples'],
                    random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'FashionMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                            per_task_memory_examples=params['per_task_memory_examples'],
                            per_task_examples = params['per_task_examples'],
                            random_class_idx = params['random_class_idx'])
    input_dim = (28, 28)
elif params['dataset'] == 'CIFAR10':
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif params['dataset'] == 'CIFAR100':        
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = params['random_class_idx'])
    input_dim = (3, 32, 32)
elif params['dataset'] in ["BiasedMNIST"]:
    benchmark = BiasedMNIST(num_tasks=params['num_tasks'],
                            per_task_memory_examples=params['per_task_memory_examples'],
                            per_task_examples = params['per_task_examples'],
                            joint = (params['method'] == "joint"),
                            random_class_idx = params['random_class_idx'])
    input_dim = (3, 28, 28)
else:
    raise NotImplementedError
class_idx = benchmark.class_idx
num_classes = len(class_idx)



[0 1 2 3 4 5 6 7 8 9]


In [3]:
from algorithms.wa import WA
from metrics import MetricCollector2
from metrics import FairMetricCollector as MetricCollector
from backbones import MLP2Layers2

backbone = MLP2Layers2(
    input_dim=input_dim, 
    hidden_dim_1=256, 
    hidden_dim_2=256, 
    output_dim=num_classes,
    class_idx=class_idx,
    config=params
    ).to(params['device'])
algorithm = WA(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [4]:
# from trainers import ContinualTrainer
# from trainers.fair_trainer import FairContinualTrainer2 as ContinualTrainer
from trainers.baselines import BaseContinualTrainer as ContinualTrainer
trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])


In [5]:
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------
[1] Eval metrics for task 1 >> {'accuracy': 0.9751393508945428, 'loss': 0.00020905635352675797, 'std': 0.023098534568012252, 'EER': -1, 'EO': [0.10444444444444445, 0.003669724770642202], 'DP': -1, 'accuracy_s0': 1.0, 'accuracy_s1': 0.9459429153924567, 'classwise_accuracy': {0: array([933, 980]), 1: array([1133, 1135])}, 'DP_ingredients': {'class_pred_count_s0': {0: 530, 1: 590}, 'class_pred_count_s1': {1: 590, 0: 405}, 'class_pred_count': {0: 935, 1: 1180}, 'count_s0': 1120, 'count_s1': 995, 'count': 2115}}
[2] Eval metrics for task 1 >> {'accuracy': 0.9920704845814978, 'loss': 0.00014037136120576384, 'std': 0.007929515418502175, 'EER': -1, 'EO': [0.03302752293577982, 0.0], 'DP': -1, 'accuracy_s0': 1.0, 'accuracy_s1': 0.9834862385321101, 'classwise_accuracy': {1: array([1117, 1135]), 0: array([980, 980])}, 'DP_ingredients': {'class_pred_count_s0': {1: 590, 0: 530}, 'class_pred_count_s1': {1: 527, 0: 468}, 'class_pred_count': {

In [ ]:
metric_manager_callback.meters['accuracy'].get_data()

array([[0.999, 0.   , 0.   , 0.   , 0.   ],
       [0.933, 0.984, 0.   , 0.   , 0.   ],
       [0.894, 0.826, 0.993, 0.   , 0.   ],
       [0.884, 0.776, 0.901, 0.988, 0.   ],
       [0.84 , 0.682, 0.689, 0.905, 0.964]])

In [ ]:
np.mean(metric_manager_callback.meters['accuracy'].compute_overall())

0.9131171979468625

In [ ]:
[np.round(x, 3) for x in metric_manager_callback.meters['EER'].compute_overall()]

[0.0, 0.025, 0.059, 0.057, 0.104]

In [ ]:
np.mean(metric_manager_callback.meters['EER'].compute_overall())

0.04924443475828506

In [ ]:
[np.round(x, 3) for x in metric_manager_callback.meters['std'].compute_overall()]

[0.001, 0.027, 0.074, 0.077, 0.122]

In [ ]:
np.mean(metric_manager_callback.meters['std'].compute_overall())

0.06027322136485429

In [ ]:
print(f"accuracy:{np.mean(metric_manager_callback.meters['accuracy'].compute_overall())}")
print(f"EER:{np.mean(metric_manager_callback.meters['EER'].compute_overall())}")
print(f"std:{np.mean(metric_manager_callback.meters['std'].compute_overall())}")


accuracy:0.9131171979468625
EER:0.04924443475828506
std:0.06027322136485429
